In [1]:
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

import pandas as pd

In [10]:
# transactions
q = '''select
            id,
            date,
            form,
            gift_assist_count
        from transactions
        where
            status='A' and
            recurring=0 and
            year>=2019'''
trans_onetime = redshift_query_read(q, schema='production')

In [11]:
# recurring: count, mean, median
q = '''select
            t.id,
            t.date,
            t.form,
            t.recurring,
            t.gift_assist_count
        from transactions as t
        inner join (select
                        recurring,
                        min(date) as min_date
                    from transactions
                    group by recurring) as r on t.recurring=r.recurring and t.date=r.min_date
        where
            t.status='A' and
            t.recurring!=0 and
            r.min_date>=2019
        order by date asc;'''
trans_rec = redshift_query_read(q, schema='production')

In [12]:
trans_rec = trans_rec.groupby('recurring').first()

In [23]:
aggs = {
    'id': 'count',
    'gift_assist_count': 'sum'
}
trans = pd.concat([trans_onetime, trans_rec]).groupby(['form', 'date']).agg(aggs).reset_index()
trans['trans_count'] = trans['id']
trans.drop('id', axis=1, inplace=True)

In [14]:
# traffic
q = '''select
            date,
            form,
            sum(views) as views
        from googleanalytics_traffic
        where
            date>=2019 and
            form!=0
        group by date, form'''
traff = redshift_query_read(q, schema='production')

In [26]:
# merge
df = trans.merge(traff, on=['date', 'form'], how='outer').fillna(0)

In [28]:
print("{:,} entries; {:,} entries w/ >0 views".format(len(df), len(df[df['views']>0])))

5,113,155 entries; 4,583,711 entries w/ >0 views


In [32]:
df = df[df['views']>0]
df['conversion'] = df['trans_count'] / df['views']

# get forms gift assist active by month
df['month'] = df['date'].dt.to_period('M')
forms_ga = df.groupby(['form', 'month'])['gift_assist_count'].sum().reset_index()
forms_ga['ga_active'] = forms_ga['gift_assist_count'] > 0
forms_ga.drop('gift_assist_count', axis=1, inplace=True)

df = df.merge(forms_ga, on=['form', 'month'])

In [36]:
df_grpd = df.groupby(['month', 'ga_active'])['conversion'].agg(['mean', 'median']).reset_index()

df_grpd.pivot(index='month', columns='ga_active', values=['mean', 'median']).reset_index()

month      mean           median          
ga_active              False      True  False      True
0          2019-01  0.050950  0.125213    0.0  0.000000
1          2019-02  0.050572  0.191673    0.0  0.000000
2          2019-03  0.048806  0.192192    0.0  0.000000
3          2019-04  0.047956  0.265539    0.0  0.000000
4          2019-05  0.058991  0.313783    0.0  0.000000
5          2019-06  0.045521  0.285931    0.0  0.000000
6          2019-07  0.038843  0.300636    0.0  0.000000
7          2019-08  0.080112  0.337946    0.0  0.000000
8          2019-09  0.059417  0.422331    0.0  0.000000
9          2019-10  0.057180  0.394748    0.0  0.000000
10         2019-11  0.055867  0.284484    0.0  0.000000
11         2019-12  0.075769  0.267533    0.0  0.000000
12         2020-01  0.048679  0.214181    0.0  0.000000
13         2020-02  0.051685  0.388806    0.0  0.000000
14         2020-03  0.071797  0.342450    0.0  0.000000
15         2020-04  0.076858  0.412112    0.0  0.000000
16         2020-05  0.084498  0.366060    0.0  0.000000
17         2020-06  0.054561  0.446545    0.0  0.000000
18         2020-07  0.054544  0.396219    0.0  0.000000
19         2020-08  0.065868  0.518862    0.0  0.000000
20         2020-09  0.074010  0.595006    0.0  0.010444
21         2020-10  0.072084  0.419324    0.0  0.000000
22         2020-11  0.074121  0.329978    0.0  0.000000
23         2020-12  0.102508  0.408689    0.0  0.000000
24         2021-01  0.054227  0.252034    0.0  0.000000
25         2021-02  0.060033  0.329543    0.0  0.000000
26         2021-03  0.064087  0.360029    0.0  0.000000
27         2021-04  0.067408  0.392893    0.0  0.000000
28         2021-05  0.058189  0.351874    0.0  0.000000
29         2021-06  0.055068  0.281277    0.0  0.000000
30         2021-07  0.040501  0.293513    0.0  0.000000
31         2021-08  0.052612  0.410469    0.0  0.000000
32         2021-09  0.063911  0.565917    0.0  0.005556
33         2021-10  0.062989  0.488324    0.0  0.000000
34         2021-11  0.075653  0.416648    0.0  0.000000
35         2021-12  0.071704  0.400562    0.0  0.024793
36         2022-01  0.053064  0.372287    0.0  0.000000
37         2022-02  0.049408  0.509611    0.0  0.000000
38         2022-03  0.057201  0.481342    0.0  0.000000
39         2022-04  0.084801  0.381700    0.0  0.000000
40         2022-05  0.054376  0.319418    0.0  0.000000
41         2022-06  0.052894  0.270498    0.0  0.000000
42         2022-07  0.050317  0.312389    0.0  0.000000
43         2022-08  0.051442  0.322968    0.0  0.000000
44         2022-09  0.059251  0.470521    0.0  0.000000
45         2022-10  0.071796  0.481262    0.0  0.000000
46         2022-11  0.066664  0.410028    0.0  0.000000
47         2022-12  0.075687  0.353379    0.0  0.002825
48         2023-01  0.039989  0.403956    0.0  0.000000
49         2023-02  0.073024  0.702720    0.0  0.000000
50         2023-03  0.068295  0.491085    0.0  0.000000